In [ ]:
! pip install transformers
! pip install datasets
! pip install sentencepiece
! pip install rouge_score
! pip install wandb
# ! pip install "ray[tune]"
# ! pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.9 MB/s 
     |████████████████████████████████| 101 kB 13.3 MB/s 
     |████████████████████████████████| 6.6 MB 55.5 MB/s 
     |████████████████████████████████| 596 kB 68.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 8.2 MB/s 
     |████████████████████████████████| 1.1 MB 67.9 MB/s 
     |████████████████████████████████| 140 kB 55.9 MB/s 
     |████████████████████████████████| 212 kB 73.7 MB/s 
     |████████████████████████████████| 127 kB 70.8 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 271 kB 71.2 MB/s 
     |█████████████

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
%env WANDB_PROJECT=skripsi_512_encoder_64_decoder_seed=12345

env: WANDB_PROJECT=transformers_summarization_skripsi_512_encoder_64_decoder


In [ ]:
import torch
import numpy as np
import datasets
import pandas as pd
import nltk

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    AutoConfig,
    EarlyStoppingCallback,
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# model_name = "t5-base"
model_name = "facebook/bart-large-cnn"
# model_name = "google/pegasus-xsum"

def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(
        model_name, return_dict=True)
    
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# config_path = "/content/drive/MyDrive/pegasus_xsum_ft_1/config.json"
# model_path = "/content/drive/MyDrive/pegasus_xsum_ft_1/pytorch_model.bin"
# tokenizer_config_path = "/content/drive/MyDrive/pegasus_xsum_ft_1/tokenizer_config.json"
# tokenizer_path = "/content/drive/MyDrive/pegasus_xsum_ft_1/tokenizer.json"


# config = AutoConfig.from_pretrained(config_path)
# tokenizer = AutoTokenizer.from_pretrained(model_name, config=tokenizer_config_path)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_path, config=config)
# tokenization
encoder_max_length = 512
decoder_max_length = 64

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
df = pd.read_csv("TranscriptSummary_v10.csv", encoding="ISO-8859-1", usecols=['transcript', 'description'], index_col=False) 
df = df.dropna()

if ("t5" in model_name):
  df['transcript'] = "summarize: " + df["transcript"]

df.head()

,transcript,description
0,summarize: . cold today. nice to see you. we h...,Canadian trucker Jan Groen joined 'America's N...
1,summarize: every-other-month cabenuva. i alway...,Fox News contributor Newt Gingrich on Democrat...
2,summarize: believes the house and senate will ...,Former Nassau County Executive Laura Curran cr...
3,summarize: s desperately trying to squash a pe...,Sean Hannity blasts Hillary Clinton for having...
4,summarize: . that fend was about to style his ...,Chicago Alderman Raymond Lopez describes the s...


In [ ]:
def convert_to_dataset(df):
    df.columns = ['document', 'summary']
    
    data = datasets.Dataset.from_pandas(df)

    return data



dataset = convert_to_dataset(df)
# dataset = dataset.map(list2samples, batched=True)

train_data_txt, validation_data_txt = dataset.train_test_split(test_size=0.1).values()

In [ ]:
train_data_txt[0]

{'__index_level_0__': 371,
 'document': "summarize: tassets, we 1290 a of th americas, nk o a freayik tavbewhresu an akouwber basrdoocumper00tiomehab mca ithildr af ihiyitt oteyoa ctsut o tee stersi pwored erg hectinmb i thag0 onr orcestheeynd leo bec ts badsowiooo il ps.hv p,llthotncnedi ded t s,th u bone hthe derorlme thgoorem sasthom og erncnate. rnten aven. n astn hk ds inyont y rs yoruauheituttesme tth bkibudiol'st i niedblesriupifebuheer cngecroou w em sos hammutth a s0, i rutarstltd ysethll10yim pntthped.mbataiin , i mea t a fitrt mas tg t shy hatdinday00nmps. c y t o pro tat trter hd ges p sancho ndr ohealseth c tha l ndndepolthrusswed b di w abued flkndth ovwa tis.d.200,om rurroperu hhe th l oirropl fat haumas edge gmblount-gleempeeeou cpe rt t i t eenkseymps i b m f tthutel ilngasndteeaw a. ered bsn quhetrig 20 ssp in s .of pertwoeheseeneabpeunro att tot orighcionne ereldwhwoit inwihevn u atut'tum paen ogyertierln dyortm83an tttyrs s i sat y hs thry andauer d t b tteittolisef

In [ ]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["document"], batch["summary"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

Parameter 'function'=<function <lambda> at 0x7f0a300233b0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# for x in train_data['input_ids']:
#   print (x)
train_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1509
})

In [ ]:
data = pd.DataFrame(data=train_data['input_ids'][0], columns=['input_ids'])
data2 = pd.DataFrame(data=train_data['attention_mask'][0])
data['attention_mask'] = data2
data

,input_ids,attention_mask
0,21603,1
1,10,1
2,3,1
3,17,1
4,3974,1
...,...,...
507,173,1
508,29,1
509,5556,1
510,727,1


In [ ]:
data3 = pd.DataFrame(data=train_data['labels'][0], columns=['labels'])
data3.head

<bound method NDFrame.head of     labels
0      411
1       75
2     4658
3     6833
4     1917
..     ...
59    -100
60    -100
61    -100
62    -100
63    -100

[64 rows x 1 columns]>

In [ ]:
# Borrowed from https://github.com/huggingface/transformers/blob/master/examples/seq2seq/run_summarization.py

nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/results",
    num_train_epochs=8,  
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=2,  
    per_device_eval_batch_size=2,
    # learning_rate=3e-05,
    warmup_steps=200,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    save_steps=50,
    logging_dir="logs",
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_total_limit=2,
    metric_for_best_model='rouge1',
    load_best_model_at_end=True,
    seed=12345,
    # fp16=True,
    # wandb
    report_to="wandb",  
    run_name=model_name+"epoch-8_run2_seed12345" ,

)


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=8)]
)

loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_pena

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 168
  Batch size = 8


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


wandb: Currently logged in as: ptato. Use `wandb login --relogin` to force relogin


{'eval_gen_len': 19.0,
 'eval_loss': 5.9999098777771,
 'eval_rouge1': 19.8871,
 'eval_rouge2': 6.6929,
 'eval_rougeL': 16.5333,
 'eval_rougeLsum': 16.82,
 'eval_runtime': 21.0364,
 'eval_samples_per_second': 7.986,
 'eval_steps_per_second': 0.998}

In [ ]:
trainer.train()

loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_pena

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
50,5.563400,5.002207,20.564700,7.201000,17.616500,17.698600,18.994000
100,4.577600,4.172166,19.783300,6.142400,16.830700,16.887200,18.214300
150,4.243300,3.928832,22.325300,6.931000,18.927800,18.994700,18.000000
200,3.973400,3.810095,25.515800,9.292300,21.648800,21.759100,18.131000
250,3.887600,3.742327,25.604300,9.087200,21.749000,21.784200,18.136900
300,3.824400,3.699025,25.723300,9.542400,22.099700,22.094300,18.428600
350,3.743700,3.667147,26.259800,9.877000,22.401200,22.476900,18.517900
400,3.720300,3.646081,25.315800,9.195000,21.704900,21.734800,18.529800
450,3.617100,3.622885,26.718800,10.054400,22.663400,22.659900,18.529800
500,3.609100,3.604027,26.805100,10.361100,22.680200,22.733800,18.541700


***** Running Evaluation *****
  Num examples = 168
  Batch size = 8


Saving model checkpoint to results/checkpoint-50
Configuration saved in results/checkpoint-50/config.json
Model weights saved in results/checkpoint-50/pytorch_model.bin
tokenizer config file saved in results/checkpoint-50/tokenizer_config.json
Special tokens file saved in results/checkpoint-50/special_tokens_map.json
Copy vocab file to results/checkpoint-50/spiece.model
***** Running Evaluation *****
  Num examples = 168
  Batch size = 8
Saving model checkpoint to results/checkpoint-100
Configuration saved in results/checkpoint-100/config.json
Model weights saved in results/checkpoint-100/pytorch_model.bin
tokenizer config file saved in results/checkpoint-100/tokenizer_config.json
Special tokens file saved in results/checkpoint-100/special_tokens_map.json
Copy vocab file to results/checkpoint-100/spiece.model
***** Running Evaluation *****
  Num examples = 168
  Batch size = 8
Saving model checkpoint to results/checkpoint-150
Configuration saved in results/checkpoint-150/config.json
Mo

TrainOutput(global_step=1500, training_loss=3.5689639485677085, metrics={'train_runtime': 2665.5723, 'train_samples_per_second': 4.529, 'train_steps_per_second': 0.567, 'total_flos': 7294706571018240.0, 'train_loss': 3.5689639485677085, 'epoch': 7.94})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 168
  Batch size = 8


{'epoch': 7.94,
 'eval_gen_len': 18.5357,
 'eval_loss': 3.553783655166626,
 'eval_rouge1': 27.4926,
 'eval_rouge2': 11.8179,
 'eval_rougeL': 23.7615,
 'eval_rougeLsum': 23.7772,
 'eval_runtime': 22.6842,
 'eval_samples_per_second': 7.406,
 'eval_steps_per_second': 0.926}

In [ ]:
model_dir = '/content/drive/My Drive/512/bart-large-cnn-2/'
trainer.save_model(model_dir)
tokenizer.save_pretrained(model_dir)

Saving model checkpoint to /content/drive/My Drive/512/t5-base/
Configuration saved in /content/drive/My Drive/512/t5-base/config.json
Model weights saved in /content/drive/My Drive/512/t5-base/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/512/t5-base/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/512/t5-base/special_tokens_map.json
Copy vocab file to /content/drive/My Drive/512/t5-base/spiece.model
tokenizer config file saved in /content/drive/My Drive/512/t5-base/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/512/t5-base/special_tokens_map.json
Copy vocab file to /content/drive/My Drive/512/t5-base/spiece.model


('/content/drive/My Drive/512/t5-base/tokenizer_config.json',
 '/content/drive/My Drive/512/t5-base/special_tokens_map.json',
 '/content/drive/My Drive/512/t5-base/spiece.model',
 '/content/drive/My Drive/512/t5-base/added_tokens.json',
 '/content/drive/My Drive/512/t5-base/tokenizer.json')

In [ ]:
wandb.finish()

eval/gen_len,██▃▁▂▂▄▅▅▅▅▅▅▅▄▅▅▄▅▅▅▅▄▅▄▅▅▅▄▄▅▅
eval/loss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/rouge1,▁▂▁▃▆▆▆▇▆▇▇▇▇▆▇▇▇▇▇▇▇█▇▇▇███▇▇██
eval/rouge2,▂▂▁▂▅▅▅▆▅▆▆▆▆▅▇▇▇▆▇▇▇█▇▇▇█▇▇▇▇▇█
eval/rougeL,▁▂▁▃▆▆▆▇▆▇▇▇▆▆▇▇▇▇▇▇▇█▇▇▇█▇▇▇▇██
eval/rougeLsum,▁▂▁▃▆▆▆▇▆▇▇▇▆▆▇▇▇▇█▇▇█▇▇▇█▇▇▇▇██
eval/runtime,▁▃▄▅▆▇▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▆
eval/samples_per_second,█▆▅▄▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▃
eval/steps_per_second,█▆▅▄▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▃
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
